In [1]:
import pandas as pd
import  numpy as np
import pymongo
from pymongo import MongoClient
from time import gmtime, strftime
from datetime import datetime



# Current Date

In [2]:
date = datetime.now().strftime('%Y-%m-%d')
date

'2020-06-11'

# https://www.coronatracker.com/country/india/

In [3]:
df = pd.read_json("https://api.coronatracker.com/v4/analytics/newcases/country?countryCode=IN&startDate=2020-04-10&endDate=2020-06-10")

In [4]:
df.tail()

,country,country_code,last_updated,new_deaths,new_infections,new_recovered
55,India,IN,2020-06-05T00:01:00.000Z,286,9471,4781
56,India,IN,2020-06-06T00:02:00.000Z,297,10438,5464
57,India,IN,2020-06-07T00:04:00.000Z,261,10884,5153
58,India,IN,2020-06-08T00:05:00.000Z,266,8422,5247
59,India,IN,2020-06-09T00:03:00.000Z,277,10218,5572


# Saving India data in csv file

In [5]:
India = df.tail(1)
India
India.to_csv(date + "_India.csv")

# https://www.covid19india.org/

In [6]:
df2 = pd.read_json("https://api.covid19india.org/v3/min/data.min.json")
df2["MH"][3]

{'confirmed': 94041, 'deceased': 3438, 'recovered': 44516, 'tested': 595282}

In [7]:
df2["MH"][3]["confirmed"]

94041

# Maharashtra New cases 

In [8]:


New_cases_in_Maharashtra = df2["MH"][0]["confirmed"]
Recoveries_in_Maharashtra =  df2["MH"][0]["recovered"]
Deaths_in_Maharashtra = df2["MH"][0]["deceased"]
Tested_in_Maharashtra = df2["MH"][0]["tested"]
Maharashtra_New = [New_cases_in_Maharashtra,Recoveries_in_Maharashtra,Deaths_in_Maharashtra,Tested_in_Maharashtra]
Maharashtra_New
Maharashtra_New = pd.DataFrame(Maharashtra_New)
Maharashtra_New  = Maharashtra_New.transpose()
Maharashtra_New.columns = ["New_cases_in_Maharashtra","Recoveries_in_Maharashtra","Deaths_in_Maharashtra","Tested_in_Maharashtra"]
Maharashtra_New.head()



,New_cases_in_Maharashtra,Recoveries_in_Maharashtra,Deaths_in_Maharashtra,Tested_in_Maharashtra
0,3254,1877,149,15988


# Maharashtra  cases 

In [9]:


Cases = df2["MH"][3]["confirmed"]
Recoveries =  df2["MH"][3]["recovered"]
Deaths = df2["MH"][3]["deceased"]
Tested = df2["MH"][3]["tested"]
Maharashtra = [date,Cases,Recoveries,Deaths,Tested]
Maharashtra
Maharashtra = pd.DataFrame(Maharashtra)
Maharashtra  = Maharashtra.transpose()
Maharashtra.columns = ["Date","Cases","Recoveries","Deaths","Tested"]
Maharashtra.head()



,Date,Cases,Recoveries,Deaths,Tested
0,2020-06-11,94041,44516,3438,595282


In [10]:
df2["KA"][0]

{'confirmed': 120, 'deceased': 3, 'recovered': 257, 'tested': 8249}

In [11]:
df2["KA"][3]

{'confirmed': 6041,
 'deceased': 69,
 'migrated': 2,
 'recovered': 2862,
 'tested': 408506}


# Karnataka New cases 

In [12]:


New_cases_in_Karnataka = df2["KA"][0]["confirmed"]
Recoveries_in_Karnataka =  df2["KA"][0]["recovered"]
Deaths_in_Karnataka = df2["KA"][0]["deceased"]
Tested_in_Karnataka = df2["KA"][0]["tested"]
Karnataka_New = [New_cases_in_Karnataka,Recoveries_in_Karnataka,Deaths_in_Karnataka,Tested_in_Karnataka]
Karnataka_New
Karnataka_New = pd.DataFrame(Karnataka_New)
Karnataka_New  = Karnataka_New.transpose()
Karnataka_New.columns = ["New_cases_in_Karnataka","Recoveries_in_Karnataka","Deaths_in_Karnataka","Tested_in_"]
Karnataka_New.head()



,New_cases_in_Karnataka,Recoveries_in_Karnataka,Deaths_in_Karnataka,Tested_in_
0,120,257,3,8249



# Karnataka  cases 

In [13]:


Cases = df2["KA"][3]["confirmed"]
Recoveries =  df2["KA"][3]["recovered"]
Deaths = df2["KA"][3]["deceased"]
Tested = df2["KA"][3]["tested"]
Karnataka = [Cases,Recoveries,Deaths,Tested]
Karnataka
Karnataka = pd.DataFrame(Karnataka )
Karnataka  = Karnataka .transpose()
Karnataka.columns = ["cases","recoveries","deaths","tested"]
Karnataka.head()



,cases,recoveries,deaths,tested
0,6041,2862,69,408506


In [14]:
df = pd.concat([Maharashtra,Maharashtra_New,Karnataka,Karnataka_New],axis =1)

In [15]:
df

,Date,Cases,Recoveries,Deaths,Tested,New_cases_in_Maharashtra,Recoveries_in_Maharashtra,Deaths_in_Maharashtra,Tested_in_Maharashtra,cases,recoveries,deaths,tested,New_cases_in_Karnataka,Recoveries_in_Karnataka,Deaths_in_Karnataka,Tested_in_
0,2020-06-11,94041,44516,3438,595282,3254,1877,149,15988,6041,2862,69,408506,120,257,3,8249


# Saving Maharashtra & Karnataka data in csv file

In [16]:
df = pd.concat([Maharashtra,Maharashtra_New,Karnataka,Karnataka_New],axis =1)
df
df.to_csv(date + "_State.csv",index = False)

# uploading state data in Mongodb

In [17]:
client = MongoClient('localhost', 27017)
db = client.covid
state = db.state
state = pd.read_csv(date + "_State.csv") #csv file which you want to import
records_ = state.to_dict(orient = 'records')
result = db.state.insert_many(records_ )

# uploading India data in Mongodb

In [18]:
client = MongoClient('localhost', 27017)
db = client.covid
India = db.India
India = pd.read_csv(date + "_India.csv") #csv file which you want to import
records_ = India.to_dict(orient = 'records')
result = db.India.insert_many(records_ )